In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

!['KeyPoints HPE'](KeyPoints_HPE.jpg)

In [2]:
DATASET_PATH = "Novos Renderizados 2025"
LABELS = ["forehand", "backhand", "nenhum"]

for categoria in LABELS:
    pasta_videos = os.path.join(DATASET_PATH, categoria)
    num_videos = len(os.listdir(pasta_videos))
    print(f"Classe {categoria}: {num_videos} vídeos")

Classe forehand: 69 vídeos
Classe backhand: 46 vídeos
Classe nenhum: 57 vídeos


In [ ]:
#Inicializando o MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

db_path = "Novos Renderizados 2025"
mov_labels = {"forehand": 0, "backhand": 1, "nenhum": 2}

X_data = []
y_data = []

print("Extraindo landmarks dos vídeos...")

braços_landmarks = [11, 12, 13, 14, 15, 16]  

#Para cada item no diretório:
for categoria, label in mov_labels.items():
    pasta_videos = os.path.join(db_path, categoria)

    for video_nome in os.listdir(pasta_videos):
        video_path = os.path.join(pasta_videos, video_nome)
        cap = cv2.VideoCapture(video_path)

        landmarks_seq = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)

            if results.pose_landmarks:
                frame_landmarks = []
                for i in braços_landmarks:  #Apenas os pontos dos braços
                    lm = results.pose_landmarks.landmark[i]
                    frame_landmarks.extend([lm.x, lm.y, lm.z])  

                landmarks_seq.append(frame_landmarks)

        cap.release()

        janela = 30
        if len(landmarks_seq) < janela:
            padding = [[0] * (len(braços_landmarks) * 3)] * (janela - len(landmarks_seq))
            landmarks_seq = padding + landmarks_seq
        else:
            landmarks_seq = landmarks_seq[-janela:]

        X_data.append(landmarks_seq)
        y_data.append(label)

X_data = np.array(X_data)
y_data = np.array(y_data)

y_data = tf.keras.utils.to_categorical(y_data, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#Ajuste dos pesos para balanceamento (caso necessário)
class_weights = {0: 1.5, 1: 1.0, 2: 2.0}

print("Pesos das classes definidos manualmente:", class_weights)

print(f"Foi possível extrair os landmarks com sucesso.\nTotal de amostras: {len(X_data)}\n")
print("Treinando o LSTM...")

#Criando o modelo LSTM
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(30, len(braços_landmarks) * 3)),
    LSTM(64),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weights)

model.save("modelo_tenis_multiclasse_v2.h5")

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Modelo treinado e salvo")
print(f"Acurácia final: {accuracy * 100:.2f}%")

🔄 Extraindo landmarks dos vídeos...
Pesos das classes definidos manualmente: {0: 1.5, 1: 1.0, 2: 2.0}
Landmarks extraídos! Total de amostras: 172
Treinando o modelo LSTM...
Epoch 1/40


c:\Users\rodri\Documents\Codigos\IT_Folder\.tennis_it\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.3104 - loss: 1.6128 - val_accuracy: 0.3429 - val_loss: 1.1226
Epoch 2/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3033 - loss: 1.5656 - val_accuracy: 0.3429 - val_loss: 1.1330
Epoch 3/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.4002 - loss: 1.5456 - val_accuracy: 0.6000 - val_loss: 1.0544
Epoch 4/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5142 - loss: 1.5187 - val_accuracy: 0.5714 - val_loss: 0.9671
Epoch 5/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5953 - loss: 1.4277 - val_accuracy: 0.6857 - val_loss: 0.7516
Epoch 6/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6219 - loss: 1.2193 - val_accuracy: 0.6857 - val_loss: 0.5946
Epoch 7/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6420 - loss: 1.1776 - val_accuracy: 0.7429 - val_loss: 0.6406
Epoch 8/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6535 - loss: 1.0095 - val_accuracy: 0.6857 - val_loss: 0.6495
Epoch 9/40

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8821 - loss: 0.3169
Modelo treinado e salvo
Acurácia final: 88.57%
